In [1]:
# imports
import folium
from haversine import haversine, Unit
from pprint import pprint
import numpy as np
import time as t
from timeit import default_timer as timer
from datetime import timedelta
from datetime import datetime, timedelta
import pandas as pd
from decimal import Decimal, ROUND_UP
import osmnx as ox
import networkx as nx
import sys
import io
from PIL import Image, ImageDraw, ImageFont
import folium.plugins as plugins
from IPython.display import display, clear_output
import os
import ipywidgets as widgets


np.set_printoptions(threshold=sys.maxsize)

ox.config(log_console=False, use_cache=True)

/usr/local/lib/python3.10/dist-packages/osmnx/utils.py:192: UserWarning: The `utils.config` function is deprecated and will be removed in a future release. Instead, use the `settings` module directly to configure a global setting's value. For example, `ox.settings.log_console=True`.
  warnings.warn(


In [2]:
# Khi - HQ
Khi_latlng = (24.853803, 67.035395)

# Saleh Pat
slp_latlng = (27.519, 69.044)

# Hyd - Cantt
hyd_latlang = (25.399066, 68.359492)

# Bdn - Cantt
bdn_latlang = (24.657308, 68.859181)


In [3]:
# location where you want to find your route
place     = 'Sindh, Pakistan'
# find shortest route based on the mode of travel
mode      = 'drive'        # 'drive', 'bike', 'walk'
# find shortest path based on distance or time
optimizer = 'length'        # 'length','time'
print('intersting')
if os.path.exists('sindh_graph.graphml'):
    print('here')
    graph = ox.load_graphml('sindh_graph.graphml')
    print('done')
else:
    # Get all nodes in a certain area which are permissible for driving
    graph = ox.graph_from_place(place, network_type = mode, simplify = False)
    ox.save_graphml(graph, 'sindh_graph.graphml')

intersting
here
done


In [18]:
def init_folium_map():
    
    basic_map = folium.Map(location= (26.277998, 68.288195), 
                           zoom_start=7,
                           control_scale= True)
    
    folium.TileLayer('Stamen Terrain').add_to(basic_map)
    folium.TileLayer('Stamen Toner').add_to(basic_map)
    folium.TileLayer('Stamen Water Color').add_to(basic_map)
    folium.TileLayer('cartodbpositron').add_to(basic_map)
    folium.TileLayer('cartodbdark_matter').add_to(basic_map)
    
    return basic_map

In [19]:
def zoom_path(path_list):
    
    interpol_path_list = []
    
    for i in range(len(path_list)-1):
        
        # Interpolation of orignal point from line formula
        # increase np.linspace value for refinement
        
        # since end point will be catered for in the next iteration, endpoint = False
        # x = x1 + (x2-x1) * t
        # y = y1 + (y2-y1) * t
        
        for t in np.linspace(0, 1, 10, endpoint= False ):
        
            interpol_val_x = path_list[i][0] + (path_list[i+1][0] - path_list[i][0]) * t
            interpol_val_y = path_list[i][1] + (path_list[i+1][1] - path_list[i][1]) * t
            interpol_val = (interpol_val_x, interpol_val_y)
            interpol_path_list.append(interpol_val)
    
    return interpol_path_list

In [20]:
def get_path_coordinates(cvy_start, cvy_dest):
    
    path_list = []
    if cvy_start == 'Karachi':
        # Khi - HQ
        start_latlng = Khi_latlng
        
        if cvy_dest == 'Hyderabad':
            end_latlng = hyd_latlang
        
        if cvy_dest == 'Badin':
            end_latlng = bdn_latlang
            
        if cvy_dest == 'Saleh Pat':
            end_latlng = slp_latlng
        
        
    if cvy_start == 'Hyderabad':
        # Khi - HQ
        start_latlng = hyd_latlang
        
        if cvy_dest == 'Karachi':
            end_latlng = Khi_latlng
        
        if cvy_dest == 'Badin':
            end_latlng = bdn_latlang
            
        if cvy_dest == 'Saleh Pat':
            end_latlng = slp_latlng
            
    if cvy_start == 'Badin':
        # Khi - HQ
        start_latlng = bdn_latlang
        
        if cvy_dest == 'Hyderabad':
            end_latlng = hyd_latlang
        
        if cvy_dest == 'Karachi':
            end_latlng = Khi_latlng
            
        if cvy_dest == 'Saleh Pat':
            end_latlng = slp_latlng
        
    if cvy_start == 'Saleh Pat':
        # Khi - HQ
        start_latlng = slp_latlng
        
        if cvy_dest == 'Hyderabad':
            end_latlng = hyd_latlang
        
        if cvy_dest == 'Badin':
            end_latlng = bdn_latlang
            
        if cvy_dest == 'Karachi':
            end_latlng = Khi_latlng

    
    # find the nearest node to the start location
    orig_node = ox.distance.nearest_nodes(graph, start_latlng[0], start_latlng[1])
    
    # find the nearest node to the end location
    dest_node = ox.distance.nearest_nodes(graph, end_latlng[0], end_latlng[1])
    
    #  find the shortest path
    shortest_route = nx.shortest_path(graph,
                                      orig_node,
                                      dest_node,
                                      weight=optimizer)
    
    for shortlisted_node in shortest_route:
        path_list.append((graph.nodes[shortlisted_node]["y"], 
                          graph.nodes[shortlisted_node]["x"]))
    
    interpol_path_list = zoom_path(path_list)
    
    return interpol_path_list

In [21]:
def get_input_distance(start_time, user_time, speed):
        
    t1 = datetime.strptime(str(start_time)[0:2] + ':' + str(start_time)[2:4], '%H:%M')
    t2 = datetime.strptime(str(user_time)[0:2] + ':' + str(user_time)[2:4], '%H:%M')
    
    if t2 > t1:
        td = t2 - t1
        time_spent = td.total_seconds()/3600
    else:
        time_spent = 0

    dist_now = speed*time_spent
        
    return dist_now


In [22]:
def get_dist_coordinates(coordinates):
    
    cumulative_dist = 0
    dist_list = []
    
    for index in range(len(coordinates)):
        if index:
            dist_one = haversine(coordinates[index-1], coordinates[index])
            cumulative_dist += dist_one
            dist_list.append(cumulative_dist)
        else:
            dist_one = 0
            dist_list.append(dist_one)
    
    return dist_list


In [23]:
def get_relative_distance(user_distance, dist_coordinates):
    
    relative_dist = []

    for dist in dist_coordinates:
        
        temp = abs(dist - user_distance)
        relative_dist.append(temp)
        
    return relative_dist

In [24]:
def create_path(path, cvy_color, folium_map):
# expects a list of tuples containing path coordinates ... [(1,1),(2,2),(3,3)...]
    
    folium.PolyLine(path,
                color= cvy_color,
                weight=3,
                opacity=0.7).add_to(folium_map)

In [25]:
def draw_point_circle(x1y1, x2y2, circle_color, fg_circles):
    
    x1 = x1y1[0]
    y1 = x1y1[1]
    
    x2 = x2y2[0]
    y2 = x2y2[1]
    
    
    diameter = haversine(x1y1, x2y2)*1000
    radius = (diameter / 2)
    
    xC, yC = (x1 + x2) / 2.0, (y1 + y2) / 2.0
    
    folium.Circle(location= (xC, yC), 
                  radius= radius,
                  color= 'yellow',
                  fill_color= circle_color).add_to(fg_circles)
    
#     folium.LayerControl().add_to(basic_map)

In [26]:
def find_conflictions(cvy_inputs_df):
    
    confliction_points = []
    conflict_cvys = []
    
    
    for index, rows_outer in cvy_inputs_df.iterrows():
        # only if vehicles exist - i.e. reached destination or never begun
        veh_conf = []
        if rows_outer['Vehicle Coordinates']:
            for veh_index in rows_outer['Vehicle Coordinates']:
                veh_conf.append(rows_outer['Coordinates'][veh_index])
            
            confliction_points.append(veh_conf)
            
            
            
    
    for i, a in enumerate(confliction_points):
        if a:
            # list
            a = set(a)

            for j, b in enumerate(confliction_points):
                if b:
                    b = set (b)
                    if i != j:    
                        if a & b:
                            conflict_cvys.append((cvy_inputs_df['Name'][i], cvy_inputs_df['Name'][j]))
#                             print(cvy_inputs_df['Name'][i], cvy_inputs_df['Name'][j])
                            
    t = []
    for k in conflict_cvys:
        
        for m in conflict_cvys:
            if k == m:
                if tuple(reversed(k)) not in t:
                    t.append(k)
    
    return t
        
        
    
    

In [27]:
"""Read data from excel file"""

cvy_inputs_df = pd.read_excel('Inputs.xlsx', sheet_name= 'Sheet1',header= 0, converters= {'Veh Count': int,
                                                                                          'Name': str,
                                                                                          'Start': str,
                                                                                          'Destination': str,
                                                                                          'Start Time': str,
                                                                                          'Color': str,
                                                                                          'Speed': int, 
                                                                                          'Spacing': int})

start_time = cvy_inputs_df['Start Time'].min()

cvy_inputs_df = cvy_inputs_df.reindex(cvy_inputs_df.columns.tolist() + ['Coordinates',
                                                                        'Distance',
                                                                        'Relative Distance',
                                                                        'Vehicle Coordinates'], axis=1)

cvy_inputs_df['Coordinates'] = 0
cvy_inputs_df['Distance'] = 0
cvy_inputs_df['Relative Distance'] = 0
cvy_inputs_df['Vehicle Coordinates'] = 0


cvy_inputs_df['Coordinates'] = cvy_inputs_df['Coordinates'].astype('object')
cvy_inputs_df['Distance'] = cvy_inputs_df['Distance'].astype('object')
cvy_inputs_df['Relative Distance'] = cvy_inputs_df['Relative Distance'].astype('object')
cvy_inputs_df['Vehicle Coordinates'] = cvy_inputs_df['Relative Distance'].astype('object')
    
cvy_inputs_df

,Veh Count,Name,Start,Destination,Start Time,Color,Speed,Spacing,Coordinates,Distance,Relative Distance,Vehicle Coordinates
0,80,37 FF,Karachi,Hyderabad,0800,red,50,200,0,0,0,0
1,80,63 Baloch,Hyderabad,Saleh Pat,0800,black,50,200,0,0,0,0
2,80,22 Punjab,Karachi,Saleh Pat,0900,red,50,200,0,0,0,0
3,80,47 AK,Hyderabad,Badin,0900,black,50,200,0,0,0,0
4,80,1 Sindh,Karachi,Hyderabad,1000,red,50,200,0,0,0,0
5,80,3 Punjab,Hyderabad,Saleh Pat,1000,black,50,200,0,0,0,0
6,80,1 NLI,Karachi,Saleh Pat,1100,red,50,200,0,0,0,0
7,80,23 SAMB,Hyderabad,Badin,1100,black,50,200,0,0,0,0
8,80,75 Medium,Karachi,Hyderabad,1200,red,50,200,0,0,0,0
9,80,42 EME,Hyderabad,Saleh Pat,1200,black,50,200,0,0,0,0


In [28]:
for index, rows in cvy_inputs_df.iterrows():
    
    # get coordinates based on route and append to dataframe as new column 'Coordinates'
    path = get_path_coordinates(rows['Start'], rows['Destination'])
    cvy_inputs_df.at[index, 'Coordinates'] = path

In [29]:
dt = datetime.strptime('26 Sep 2012', '%d %b %Y')
time_list =[]
for hr in range(8, 24):
    for mn in range(0, 60, 30):
        newdatetime = dt.replace(hour=hr, minute=mn)
        time_list.append(newdatetime)

time_list_str = []
for tdate in time_list:
    time_list_str.append("{:02d}{:02d}".format(tdate.hour, tdate.minute))
    
user_time_options = widgets.SelectionSlider(
    options=time_list_str,
    value='0800',
    description='Convoy Time',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True
)

In [30]:
def main(user_time='1400'):

#     clear_output(wait= False)

#     print(user_time)
    st_time = timer()
    
    basic_map = init_folium_map()
    fg_circles = folium.FeatureGroup(name= 'circleLayer', overlay= True)


    # get distances based on coordinates and append to dataframe as new column 'Distance'
    for index, rows in cvy_inputs_df.iterrows():

        dist_temp = get_dist_coordinates(rows['Coordinates'])
        cvy_inputs_df.at[index, 'Distance'] = dist_temp
    

    for index, rows in cvy_inputs_df.iterrows():

        # converts time into distance travelled along a path
        current_dist = get_input_distance(rows['Start Time'], user_time, rows['Speed'])
        cvy_inputs_df.at[index, 'Current Head Distance'] = current_dist


    # find shortest distance of user_distance - rows['Distance']
    for index, rows in cvy_inputs_df.iterrows():

        relative_dist = get_relative_distance(rows['Current Head Distance'], rows['Distance'])
        cvy_inputs_df.at[index, 'Relative Distance'] = relative_dist
#         print(relative_dist)
        create_path(rows['Coordinates'], rows['Color'], basic_map)
    
    # iterate over convoys
    for index, rows in cvy_inputs_df.iterrows():
        # safety check - ensure convoy has started its journey
        if rows['Current Head Distance'] != 0:
            # repeat process for every vehicle in convoy
            vehicle_index_min = []

            for vehicle_num in range(rows['Veh Count']):

                if vehicle_num == 0:
                    # first vehicle, will always be relative distance = 0
                    index_min = np.argmin(rows['Relative Distance'])
                    # mark index for vehicle
                    vehicle_index_min.append(index_min)

                    zero_val = rows['Distance'][index_min] 
                    
                    # if destination has been reached
                    if np.max(rows['Distance']) == zero_val:
                        cvy_inputs_df.at[index, 'Vehicle Coordinates']= [vehicle_index_min[0] - index]
                        break
                    
                else:
                    # iterate backwards for the rest of vehicles from zero_val of distance
                    j = 0
                    while 1:
                        # safety check - ensure we do not iterate below 0, incase vehicles are too many and
                        # coordinates are not enough from given location
                        if index_min - j >= 0:
                            # when current index has a distance greater than spacing required for nth vehicle
                            if abs(zero_val - rows['Distance'][index_min-j]) > (rows['Spacing']*vehicle_num)/1000:
                                index_min = index_min - j
                                # mark index for vehicle
                                vehicle_index_min.append(index_min)
                                break
                        else:
                            break
                        j+=1

                cvy_inputs_df.at[index, 'Vehicle Coordinates']= vehicle_index_min

            # first and last vehicle coordinates, two points to determine radius of circle
            draw_point_circle(rows['Coordinates'][vehicle_index_min[0]],
                              rows['Coordinates'][vehicle_index_min[-1]],
                              rows['Color'], fg_circles)


            for index, index_min in enumerate(vehicle_index_min):

                coordinates_min = rows['Coordinates'][index_min]

                popup_text = str(rows['Name']) + ' Distance Travelled: ' + str(round(rows['Distance'][index_min], 2)) + " KM"

                if index == 0 or index == (rows['Veh Count'] - 1):

                    folium.Marker(location= coordinates_min, 
                                  popup= popup_text,
                                  icon= folium.Icon(color=rows['Color'], 
                                                    icon_color='white', 
                                                    icon = 'fa-truck', 
                                                    prefix='fa')
                                 ).add_to(basic_map)
                else:

                    folium.CircleMarker(location= coordinates_min,
                                        radius= 3,
                                        color= 'yellow',
                                        fill= True,
                                        fill_opacity= 1).add_child(folium.Popup(popup_text)).add_to(basic_map)

        else:
            popup_text = str(rows['Name']) + ' Distance Travelled: ' + str(round(rows['Distance'][index], 2)) + " KM"

            folium.Marker(location= rows['Coordinates'][index], 
                          popup= popup_text,
                          icon= folium.Icon(color=rows['Color'], 
                                            icon_color='white', 
                                            icon = 'fa-truck', 
                                            prefix='fa')).add_to(basic_map)

    conf_check = find_conflictions(cvy_inputs_df)
    
    if conf_check:
        print('Conflictions exist')
#         W, H = (2000,1500)
#         im = Image.new("RGBA",(W,H))
#         draw = ImageDraw.Draw(im)
#         msg = 'hey'
#         w, h = draw.textsize(msg)
#         fnt = ImageFont.truetype('arial.ttf', 30)
#         draw.text((0,0), msg, font=fnt,fill=(0, 0, 0))
#         im.crop((0, 0,2*w,2*h)).save("pycoatextlogo.png", "PNG")
        
#         IFrame
        print(conf_check)
    else:
        print('No Conflictions')
    
    fg_circles.add_to(basic_map)
    plugins.Fullscreen().add_to(basic_map)
    draw = plugins.Draw(export=False)
    draw.add_to(basic_map)
    folium.LayerControl().add_to(basic_map)
    display(basic_map)
    cvy_inputs_df

In [31]:
# main()

widgets.interact(main, user_time=user_time_options)

# display(user_time_options)

interactive(children=(SelectionSlider(continuous_update=False, description='Convoy Time', options=('0800', '08…

<function __main__.main(user_time='1400')>